In [1]:
import re
import pandas as pd

In [3]:
from kg_extract.doc_handle import RuDocHandle

document = RuDocHandle('test.docx')
data = document.get_doc_structure()

In [ ]:
data

In [4]:
data.to_csv('./man.csv',index=False)

In [ ]:
set(data.proper_nouns_name.values)

In [ ]:
from kg_extract.ner import RuNer

document = RuNer('en_core_web_sm', 'test.docx')

In [ ]:
data = document.get_sentences_ner()

In [ ]:
proper_nouns = list(pd.read_csv('proper_nouns.csv')['proper_nouns_name'].values)
proper_nouns

In [ ]:
temp = 'On the downlink path, the Digital Subsystem DU receives information through the Transport Network interface and uses Baseband signal processing to encode the data as IQ modulated digital data to meet the requirements of the antenna interface (AI)'.split(' ')
set(proper_nouns).intersection(set(temp))

In [ ]:
data.to_csv('./man.csv',index=False)

In [ ]:
from kg_process.es import Ru_es

es = Ru_es("http://localhost:7474",auth=("neo4j","test"))
es.insert()

In [ ]:
es.insert()

In [7]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

doc = {
    's': '',
    'p': '',
    'o': '',
    'key_words': [],
    'timestamp': datetime.now(),
}
# res = es.index(index="test-index", id=1, body=doc)
# print(res['result'])

In [13]:
graph_data = pd.read_csv('man.csv')
for i in range(0, len(graph_data)):
    doc['s'] = graph_data['s'][i]
    doc['p'] = graph_data['p'][i]
    doc['o'] = graph_data['o'][i]
    doc['key_words'] = eval(graph_data['key_words'][i])
    doc['timestamp'] = datetime.now()
    es.index(index="doc", doc_type='Radio Application Architecture Description', body=doc)

In [39]:
# res = es.get(index="test-index", id=1)
# print(res['_source'])

# es.indices.refresh(index="test-index")

res = es.search(index="doc", doc_type='Radio Application Architecture Description', 
                body={
                        "query": {
                            "match": {
                                "o": {      
                                    "query":    "RBS 3GPP",
                                    "operator": "and"
                                }
                            }
                        }
                    }
               )
# print("Got %d Hits:" % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

In [ ]:
from pptx import Presentation

prs = Presentation('test.pptx')
prs.slides[5].export("the_new_path-file.jpg", "JPG")

# text_runs will be populated with a list of strings,
# one for each text run in presentation
# text_runs = []

# for slide in prs.slides:
#     for shape in slide.shapes:
#         if not shape.has_text_frame:
#             continue
#         for paragraph in shape.text_frame.paragraphs:
#             for run in paragraph.runs:
#                 text_runs.append(run.text)

In [ ]:
pic = prs.slides[5]
with open('mypic.jpg', 'wb') as f:
    f.write(pic.image.blob)

In [ ]:
from PIL import Image
from io import BytesIO


# Convert the corrected list to "bytes", wrap in BytesIO and ask PIL to open as PIL Image
pImage = Image.open(BytesIO(bytes(pic.background.element)))

# Now we can display it
pImage.show() 

# Or save it
pImage.save('result.jpg')

In [ ]:
import win32com
import win32com.client

Application = win32com.client.Dispatch("PowerPoint.Application")

Presentation = Application.Presentations.Open(r"C:\Users\eeeinld\PycharmProjects\test_ui\test.pptx", WithWindow=False)
Presentation.Slides[5].Export(r"C:\Users\eeeinld\PycharmProjects\test_ui\test.jpg", "JPG")
Application.Quit()